In [1]:
import pandas as pd
import numpy as np
import gspread
from datetime import date
from datetime import datetime
from datetime import datetime, timedelta
from pandas import json_normalize

import re

import sys                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
sys.path.append('../')

import zoho_crm_api_module as crm

import sys
sys.path.append('../../../zoho_data_app/')

# importing
import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:
gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar.fatoni\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '1_zOrWK1gAo-tHDsKqU6AaqVzi4xaNpOELDwtbBx5F6M'
SHEET_NAME = 'Karma Beach - LIVE'


cols = [
    "id","Created_Time", "First_Name", "Last_Name", "Email", "Birthdate", "Phone", 
    "Street", "City", "Country", "Brand", "Lead_Sub_Brand", "Lead_Source", 
    "Lead_Source_Description", "Lead_Locations", "Lead_History", "Website", 
    "Card_Number", "Card_Expired", "Do_you_live_in_Bali"
]

live_cols = [
    "Record Id", "Created Time", "First Name", "Last Name", "Email", "Birthdate", 
    "Phone", "Street", "City", "Country", "Lead Brand", "Lead Sub-Brand", 
    "Lead Source", "Lead Source Description", "Lead Locations", "Lead History", 
    "Website", "Card Number", "Card Expired", "Do you live in Bali?"
]

# Function to fix the encoding
def fix_encoding(garbled_string):
    try:
        # Encode the garbled string to bytes using 'latin1' and then decode it to 'utf-8'
        return garbled_string.encode('latin1').decode('utf-8')
    except Exception as e:
        return garbled_string  # Return the original string if there's an error


In [3]:
crm_token = crm.get_crm_token()

In [4]:
df = crm.get_kbb_vip_data(crm_token, cols)
df.fillna('empty', inplace=True)
new_df = crm.get_kbb_vip_filter(df)
new_df['id'] = 'zcrm_'+new_df['id']
new_df.rename(columns={'id': 'Record Id'}, inplace=True)
new_df['Created_Time'] = new_df['Created_Time'].str.replace('T', ' ').str.replace(r'\+\d{2}:\d{2}', '', regex=True)
new_df['Created_Time']  = pd.to_datetime(new_df['Created_Time']).dt.strftime('%d/%m/%Y')
new_df['Card_Expired']  = pd.to_datetime(new_df['Card_Expired']).dt.strftime('%d/%m/%Y')
new_df['First_Name'] = new_df['First_Name'].apply(fix_encoding)
new_df['Last_Name'] = new_df['Last_Name'].apply(fix_encoding)
m.clean_number(new_df, 'Phone')
m.replace_not_valid_to_empty(new_df, 'Phone')
# new_df = df.copy()
new_df.sort_values('Created_Time', ascending=False, inplace=True)
new_df.reset_index(drop=True, inplace=True)
count_data_awal = new_df.shape[0]
new_df

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\..\zoho_crm_api_module.py:267: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  concat_df['Lead_Locations'].replace('\[|\]|\'','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\../../../zoho_data_app\main_module.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pan

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000565024037,16/07/2025,Megan,Dunne,mdunne844@gmail.com,empty,61423469244,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700160005,16/07/2026,False
1,zcrm_3584690000565003031,16/07/2025,Liv,Wilkinson,livvyyy15@hotmail.com,empty,61459387563,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700160001,16/07/2026,False
2,zcrm_3584690000564950092,16/07/2025,Julia,Petratos,juliapetratos@hotmail.com,empty,61415571630,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700160004,16/07/2026,False
3,zcrm_3584690000564950062,16/07/2025,Vasilisa,Petruk,vasilisapetruk@gmail.com,empty,6285190185129,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700160002,16/07/2026,True
4,zcrm_3584690000564949077,16/07/2025,Lynette,Lyons,lynettelyons@outlook.com.au,empty,61420901212,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700160003,16/07/2026,False
5,zcrm_3584690000564928001,15/07/2025,Scarlett,Pope,scarlett_pope@student.kambala.nsw.edu.au,empty,61468343989,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700150003,15/07/2026,False
6,zcrm_3584690000564857001,15/07/2025,Ekaterina,Ermakova,kzebrah@gmail.com,empty,79637695633,empty,empty,Russia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700150001,15/07/2026,False
7,zcrm_3584690000564921001,15/07/2025,Belinda,Tandurella,belindatand@gmail.com,empty,61408150176,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700150002,15/07/2026,False
8,zcrm_3584690000564932001,15/07/2025,Elena,Tumalevich,vanillasgalaxy@gmail.com,empty,79175906454,empty,empty,Russia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700150004,15/07/2026,False
9,zcrm_3584690000565007001,15/07/2025,Aulia,Maharani,auliamhrn1210@gmail.com,empty,31639211607,empty,empty,Netherlands,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700150005,15/07/2026,False


In [5]:
new_df['First_Name'] = new_df['First_Name'].str.title()
new_df['Last_Name'] = new_df['Last_Name'].str.title()
new_df['Email'] = m.lowercase(new_df, 'Email')
m.clean_space(new_df, 'Email')

new_df
# new_df.to_excel(r"C:\Users\fajar.fatoni\Documents\Python\Data\kbbvip_live.xlsx", index=False)

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\../../../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000565024037,16/07/2025,Megan,Dunne,mdunne844@gmail.com,empty,61423469244,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700160005,16/07/2026,False
1,zcrm_3584690000565003031,16/07/2025,Liv,Wilkinson,livvyyy15@hotmail.com,empty,61459387563,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700160001,16/07/2026,False
2,zcrm_3584690000564950092,16/07/2025,Julia,Petratos,juliapetratos@hotmail.com,empty,61415571630,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700160004,16/07/2026,False
3,zcrm_3584690000564950062,16/07/2025,Vasilisa,Petruk,vasilisapetruk@gmail.com,empty,6285190185129,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700160002,16/07/2026,True
4,zcrm_3584690000564949077,16/07/2025,Lynette,Lyons,lynettelyons@outlook.com.au,empty,61420901212,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700160003,16/07/2026,False
5,zcrm_3584690000564928001,15/07/2025,Scarlett,Pope,scarlett_pope@student.kambala.nsw.edu.au,empty,61468343989,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700150003,15/07/2026,False
6,zcrm_3584690000564857001,15/07/2025,Ekaterina,Ermakova,kzebrah@gmail.com,empty,79637695633,empty,empty,Russia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700150001,15/07/2026,False
7,zcrm_3584690000564921001,15/07/2025,Belinda,Tandurella,belindatand@gmail.com,empty,61408150176,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700150002,15/07/2026,False
8,zcrm_3584690000564932001,15/07/2025,Elena,Tumalevich,vanillasgalaxy@gmail.com,empty,79175906454,empty,empty,Russia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700150004,15/07/2026,False
9,zcrm_3584690000565007001,15/07/2025,Aulia,Maharani,auliamhrn1210@gmail.com,empty,31639211607,empty,empty,Netherlands,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000700150005,15/07/2026,False


In [6]:
# # mencari tanggal 8 hari kebelakang
# calculate_date = datetime.now() - timedelta(days=8)
# new_cal_date = str(calculate_date.strftime("%Y-%m-%d")).split(' ')
# first_date = new_cal_date[0]

# # mencari tanggal hari ini
# today = datetime.now() 
# second_date = today.strftime("%Y-%m-%d")

# print(first_date)
# print(second_date)

In [7]:
new_df.replace('empty', '', inplace=True)
new_df

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000565024037,16/07/2025,Megan,Dunne,mdunne844@gmail.com,,61423469244,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160005,16/07/2026,False
1,zcrm_3584690000565003031,16/07/2025,Liv,Wilkinson,livvyyy15@hotmail.com,,61459387563,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160001,16/07/2026,False
2,zcrm_3584690000564950092,16/07/2025,Julia,Petratos,juliapetratos@hotmail.com,,61415571630,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160004,16/07/2026,False
3,zcrm_3584690000564950062,16/07/2025,Vasilisa,Petruk,vasilisapetruk@gmail.com,,6285190185129,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160002,16/07/2026,True
4,zcrm_3584690000564949077,16/07/2025,Lynette,Lyons,lynettelyons@outlook.com.au,,61420901212,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160003,16/07/2026,False
5,zcrm_3584690000564928001,15/07/2025,Scarlett,Pope,scarlett_pope@student.kambala.nsw.edu.au,,61468343989,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700150003,15/07/2026,False
6,zcrm_3584690000564857001,15/07/2025,Ekaterina,Ermakova,kzebrah@gmail.com,,79637695633,,,Russia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700150001,15/07/2026,False
7,zcrm_3584690000564921001,15/07/2025,Belinda,Tandurella,belindatand@gmail.com,,61408150176,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700150002,15/07/2026,False
8,zcrm_3584690000564932001,15/07/2025,Elena,Tumalevich,vanillasgalaxy@gmail.com,,79175906454,,,Russia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700150004,15/07/2026,False
9,zcrm_3584690000565007001,15/07/2025,Aulia,Maharani,auliamhrn1210@gmail.com,,31639211607,,,Netherlands,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700150005,15/07/2026,False


In [8]:
import pandas as pd
# Assuming gc, SHEET_ID, SHEET_NAME, and fix_encoding are defined

df_live = pd.DataFrame() # Initialize an empty DataFrame

try:
    # Get data from Google Sheets
    spreadsheet = gc.open_by_key(SHEET_ID)
    worksheet = spreadsheet.worksheet(SHEET_NAME)
    # get_all_records assumes the first row is a header
    rows = worksheet.get_all_records()

    # Check if any data was returned
    if not rows:
        print(f"No data found in sheet '{SHEET_NAME}' or it lacks a header row.")
        # Define expected columns for an empty DataFrame if needed downstream
        df_live = pd.DataFrame(columns=['Card Expired', 'First Name', 'Last Name'])
    else:
        # Create DataFrame from the rows
        df_live = pd.DataFrame(rows)

        # --- Process columns only if they exist ---

        # Process 'Card Expired'
        if 'Card Expired' in df_live.columns:
            # Convert 'Card Expired' to datetime, handling potential errors
            df_live['Card Expired'] = pd.to_datetime(
                df_live['Card Expired'],
                dayfirst=True,
                errors='coerce' # Convert unparseable dates to NaT (Not a Time)
            )
            # Format back to string, handling NaT values gracefully
            df_live['Card Expired'] = df_live['Card Expired'].dt.strftime('%d/%m/%Y').fillna('') # Replace NaT with empty string
        else:
            print("Warning: Column 'Card Expired' not found in the sheet.")

        # Process 'First Name'
        if 'First Name' in df_live.columns:
            df_live['First Name'] = df_live['First Name'].apply(fix_encoding)
        else:
            print("Warning: Column 'First Name' not found in the sheet.")

        # Process 'Last Name'
        if 'Last Name' in df_live.columns:
            df_live['Last Name'] = df_live['Last Name'].apply(fix_encoding)
        else:
            print("Warning: Column 'Last Name' not found in the sheet.")

except Exception as e:
    print(f"An error occurred while accessing or processing the Google Sheet: {e}")
    # Ensure df_live is an empty DataFrame in case of error during processing
    if not isinstance(df_live, pd.DataFrame) or df_live.empty:
       df_live = pd.DataFrame(columns=['Card Expired', 'First Name', 'Last Name']) # Or just pd.DataFrame()
    print("DataFrame might be empty or partially processed due to the error.")


# Display the DataFrame
print("\nFinal DataFrame contents:")
# Use display() in environments like Jupyter/Colab, otherwise use print()
try:
    display(df_live)
except NameError:
    print(df_live)



Final DataFrame contents:


,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000565024037,16/07/2025,Megan,Dunne,mdunne844@gmail.com,,61423469244,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160005,16/07/2026,FALSE
1,zcrm_3584690000565003031,16/07/2025,Liv,Wilkinson,livvyyy15@hotmail.com,,61459387563,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160001,16/07/2026,FALSE
2,zcrm_3584690000564950092,16/07/2025,Julia,Petratos,juliapetratos@hotmail.com,,61415571630,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160004,16/07/2026,FALSE
3,zcrm_3584690000564950062,16/07/2025,Vasilisa,Petruk,vasilisapetruk@gmail.com,,6285190185129,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160002,16/07/2026,TRUE
4,zcrm_3584690000564949077,16/07/2025,Lynette,Lyons,lynettelyons@outlook.com.au,,61420901212,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160003,16/07/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,zcrm_3584690000007840074,02/04/2019,Oreste,Lentidoro,lentidoro@gmail.com,,61410699788,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000300100005,10/03/2025,FALSE
9982,zcrm_3584690000008003243,02/04/2019,Debby,Lumempouw,dmsl_della@yahoo.com,,6281806204001,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000200190007,19/02/2025,FALSE
9983,zcrm_3584690000008059311,02/04/2019,Meylina,Ardiningsih,meylinaardi@gmail.com,,6281385951854,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000100200010,20/01/2025,FALSE
9984,zcrm_3584690000008362510,02/04/2019,Caslin,Liu,Caslinliu@yahoo.com.sg,,6591192897,,,Singapore,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Kandara,Other Bali DB - Purchased,https://karmagroup.com/karma-beach-vip/,2024000300240002,24/03/2025,FALSE


In [9]:
df_update = new_df.loc[~new_df['Record Id'].isin(df_live['Record Id'])].copy()
df_update

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali


In [10]:
df_update.sort_values(by=['Created_Time'], inplace=True, ascending=False)
rename_mapping = dict(zip(cols, live_cols))
df_update = df_update.rename(columns=rename_mapping)
df_update.reset_index(drop=True, inplace=True)
df_update

count_df_update = df_update.shape[0]


In [11]:
df_live

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000565024037,16/07/2025,Megan,Dunne,mdunne844@gmail.com,,61423469244,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160005,16/07/2026,FALSE
1,zcrm_3584690000565003031,16/07/2025,Liv,Wilkinson,livvyyy15@hotmail.com,,61459387563,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160001,16/07/2026,FALSE
2,zcrm_3584690000564950092,16/07/2025,Julia,Petratos,juliapetratos@hotmail.com,,61415571630,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160004,16/07/2026,FALSE
3,zcrm_3584690000564950062,16/07/2025,Vasilisa,Petruk,vasilisapetruk@gmail.com,,6285190185129,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160002,16/07/2026,TRUE
4,zcrm_3584690000564949077,16/07/2025,Lynette,Lyons,lynettelyons@outlook.com.au,,61420901212,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160003,16/07/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,zcrm_3584690000007840074,02/04/2019,Oreste,Lentidoro,lentidoro@gmail.com,,61410699788,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000300100005,10/03/2025,FALSE
9982,zcrm_3584690000008003243,02/04/2019,Debby,Lumempouw,dmsl_della@yahoo.com,,6281806204001,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000200190007,19/02/2025,FALSE
9983,zcrm_3584690000008059311,02/04/2019,Meylina,Ardiningsih,meylinaardi@gmail.com,,6281385951854,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000100200010,20/01/2025,FALSE
9984,zcrm_3584690000008362510,02/04/2019,Caslin,Liu,Caslinliu@yahoo.com.sg,,6591192897,,,Singapore,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Kandara,Other Bali DB - Purchased,https://karmagroup.com/karma-beach-vip/,2024000300240002,24/03/2025,FALSE


In [12]:
# Concatenate DataFrames
df_final = pd.concat([df_update, df_live], ignore_index=True)

# Replace 'empty' with ''
df_final.replace('empty', '', inplace=True)

# Convert 'Do you live in Bali?' to uppercase
df_final['Do you live in Bali?'] = df_final['Do you live in Bali?'].astype(str).str.upper()

# Convert 'Created Time' to datetime format
df_final['Created Time'] = pd.to_datetime(df_final['Created Time'], dayfirst=True, errors='coerce')

# Sort by 'Created Time' in descending order
df_final.sort_values('Created Time', ascending=False, inplace=True)

# Format 'Created Time' back to the desired string format
df_final['Created Time'] = df_final['Created Time'].dt.strftime('%d/%m/%Y')

df_final

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000565024037,16/07/2025,Megan,Dunne,mdunne844@gmail.com,,61423469244,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160005,16/07/2026,FALSE
3,zcrm_3584690000564950062,16/07/2025,Vasilisa,Petruk,vasilisapetruk@gmail.com,,6285190185129,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160002,16/07/2026,TRUE
4,zcrm_3584690000564949077,16/07/2025,Lynette,Lyons,lynettelyons@outlook.com.au,,61420901212,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160003,16/07/2026,FALSE
1,zcrm_3584690000565003031,16/07/2025,Liv,Wilkinson,livvyyy15@hotmail.com,,61459387563,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160001,16/07/2026,FALSE
2,zcrm_3584690000564950092,16/07/2025,Julia,Petratos,juliapetratos@hotmail.com,,61415571630,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160004,16/07/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9962,zcrm_3584690000007147790,02/04/2019,Naveen,Singhal,naveensinghal007@gmail.com,,919901134010,,Bangalore,India,Karma Group,Other,Web Enquiry,Karma Beach VIP membership sign up 2023,,"Other - Web Enquiry, Karma Experience IN - Fac...",https://karmagroup.com/karma-beach-vip/,2024000100100043,10/01/2025,FALSE
9961,zcrm_3584690000005990930,02/04/2019,Jordan,Powell,jordanpowell7@hotmail.com,,64225901649,,,New Zealand,Karma Resorts,Other,Other,Karma Beach VIP membership sign up 2023,Karma Rottnest,"Other - Past Guests, Karma Royal - Karma Royal...",https://karmagroup.com/karma-beach-vip/,2024000500240017,24/05/2025,FALSE
9960,zcrm_3584690000005948267,02/04/2019,Elycia,Martins,elycia.martins@gmail.com,,61403924069,,,Australia,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,Other - Past Guests,https://karmagroup.com/karma-beach-vip/,2024000900090001,09/09/2025,FALSE
9959,zcrm_3584690000006011328,02/04/2019,Parth,J,parthsarthi@gmail.com,,6597767716,15 D Balmoral Road 10 - / 6,Singapore,Singapore,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Kandara,Other - Past Guests,https://karmagroup.com/karma-beach-vip/,2024001200260006,26/12/2025,FALSE


In [13]:
df_live.columns
# new_df.columns

Index(['Record Id', 'Created Time', 'First Name', 'Last Name', 'Email',
       'Birthdate', 'Phone', 'Street', 'City', 'Country', 'Lead Brand',
       'Lead Sub-Brand', 'Lead Source', 'Lead Source Description',
       'Lead Locations', 'Lead History', 'Website', 'Card Number',
       'Card Expired', 'Do you live in Bali?'],
      dtype='object')

In [14]:
# Function to sanitize DataFrame
def sanitize_dataframe(df):
    # Replace invalid float values with None
    df = df.applymap(lambda x: None if isinstance(x, float) and (np.isinf(x) or np.isnan(x)) else x)
    return df

# Sanitize the DataFrame
df_final = sanitize_dataframe(df_final)
df_final['Card Number'] = df_final['Card Number'].astype(str)
df_final.drop(df_final.loc[df_final['Email'].str.contains('karmagroup.com', regex=True, flags=re.I)].index, inplace=True)
df_final

C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_10972\4224994593.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if isinstance(x, float) and (np.isinf(x) or np.isnan(x)) else x)


,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000565024037,16/07/2025,Megan,Dunne,mdunne844@gmail.com,,61423469244,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160005,16/07/2026,FALSE
3,zcrm_3584690000564950062,16/07/2025,Vasilisa,Petruk,vasilisapetruk@gmail.com,,6285190185129,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160002,16/07/2026,TRUE
4,zcrm_3584690000564949077,16/07/2025,Lynette,Lyons,lynettelyons@outlook.com.au,,61420901212,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160003,16/07/2026,FALSE
1,zcrm_3584690000565003031,16/07/2025,Liv,Wilkinson,livvyyy15@hotmail.com,,61459387563,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160001,16/07/2026,FALSE
2,zcrm_3584690000564950092,16/07/2025,Julia,Petratos,juliapetratos@hotmail.com,,61415571630,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000700160004,16/07/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9962,zcrm_3584690000007147790,02/04/2019,Naveen,Singhal,naveensinghal007@gmail.com,,919901134010,,Bangalore,India,Karma Group,Other,Web Enquiry,Karma Beach VIP membership sign up 2023,,"Other - Web Enquiry, Karma Experience IN - Fac...",https://karmagroup.com/karma-beach-vip/,2024000100100043,10/01/2025,FALSE
9961,zcrm_3584690000005990930,02/04/2019,Jordan,Powell,jordanpowell7@hotmail.com,,64225901649,,,New Zealand,Karma Resorts,Other,Other,Karma Beach VIP membership sign up 2023,Karma Rottnest,"Other - Past Guests, Karma Royal - Karma Royal...",https://karmagroup.com/karma-beach-vip/,2024000500240017,24/05/2025,FALSE
9960,zcrm_3584690000005948267,02/04/2019,Elycia,Martins,elycia.martins@gmail.com,,61403924069,,,Australia,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,Other - Past Guests,https://karmagroup.com/karma-beach-vip/,2024000900090001,09/09/2025,FALSE
9959,zcrm_3584690000006011328,02/04/2019,Parth,J,parthsarthi@gmail.com,,6597767716,15 D Balmoral Road 10 - / 6,Singapore,Singapore,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Kandara,Other - Past Guests,https://karmagroup.com/karma-beach-vip/,2024001200260006,26/12/2025,FALSE


In [15]:
# Update keseluruhan data, delete data yang lama terlebih dahulu
# clear all row before update
worksheet.clear()
worksheet.update([df_final.columns.values.tolist()] + df_final.values.tolist())

{'spreadsheetId': '1_zOrWK1gAo-tHDsKqU6AaqVzi4xaNpOELDwtbBx5F6M',
 'updatedRange': "'Karma Beach - LIVE'!A1:T9987",
 'updatedRows': 9987,
 'updatedColumns': 20,
 'updatedCells': 199740}

In [16]:
print("Data update kbb vip : "+str(count_df_update))

Data update kbb vip : 0
